# How it Works

We will be receiving a string input that specifies the carbon sequestration method to be utilized.

1. Planting trees
2. Acres of U.S. forests in one year 
3. Acres of U.S. forests preserved from conversion to cropland in one year
4. Solar Panel

For 1,2,3, we're using linear regression (algorithm) to calculate carbon offsets,
and for 4, we're using machine learning and neural network to predict carbon offsets using the Google Sunroof dataset. Refer to /data/metadata.csv for details of the dataset. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Plant trees

In [6]:
#Source: https://www.unm.edu/~jbrink/365/Documents/Calculating_tree_carbon.pdf

def tree_co2_sequestration(age, height, diameter):
    # Determine the total (green) weight of the tree
    if diameter < 11:
        W = 0.25 * diameter**2 * height
    else:
        W = 0.15 * diameter**2 * height

    # Add the weight of the root system
    total_green_weight = W * 1.2

    # Determine the dry weight of the tree
    dry_weight = total_green_weight * 0.725

    # Determine the weight of carbon in the tree
    carbon_weight = dry_weight * 0.5

    # Determine the weight of carbon dioxide sequestered in the tree
    CO2_weight = carbon_weight * 3.6663

    # Determine the weight of CO2 sequestered in the tree per year
    CO2_sequestered_per_year = CO2_weight / age
    
    # Convert the CO2 sequestered per year from pounds to metric tons
    CO2_sequestered_per_year_metric_tons = CO2_sequestered_per_year / 2204.62
    
    carbon_offset = CO2_sequestered_per_year_metric_tons

    return carbon_offset

carbon_offset = 

# Examples
print(tree_co2_sequestration(10, 15, 8))       # Calliandra calothyrsus
print(tree_co2_sequestration(10, 45, 6))       # Grevillia robusta
print(tree_co2_sequestration(2.5, 15, 3))      # Acacia angustissima
print(tree_co2_sequestration(15, 30, 12))      # Albizzia lebbek


0.017361800219538967
0.029298037870472006
0.00976601262349067
0.03125124039517014


## 2. Acres of U.S. forests in one year

In [10]:
#Source: https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references#pineforests

def annual_net_change(t, t_plus_1):
    """Calculate the annual net change in carbon stocks in metric tons"""
    return (t_plus_1 - t) * 1e6

def stock_density(stock, area):
    """Calculate the carbon stock density in metric tons/hectare"""
    return stock * 1e6 / (area * 1e3)

def net_change_per_area(net_change, area):
    """Calculate the annual net change in carbon stock per area in metric tons/hectare/year"""
    return net_change / (area * 1e3)

def sequestered_per_acre(net_change_per_acre):
    """Calculate the carbon sequestered in one year by one acre in metric tons CO2/acre/year"""
    return net_change_per_acre * (44 / 12)

# Given data in MMT and thousand hectares
stock_2019 = 58007  # MMT C
stock_2020 = 58156  # MMT C
area = 282061  # in thousand hectares

# Step 1: Calculate annual net change in metric tons
net_change = annual_net_change(stock_2019, stock_2020)

# Step 2: Calculate carbon stock density and annual net change per area in metric tons/hectare and metric tons/hectare/year respectively
density_2020 = stock_density(stock_2020, area)
net_change_per_area_2020 = net_change_per_area(net_change, area)

# Conversion Factor for Carbon Sequestered in One Year by 1 Acre of Average U.S. Forest
net_change_per_acre = net_change_per_area_2020 / 2.47105  # converting hectare to acre
sequestered_per_acre_2020 = sequestered_per_acre(net_change_per_acre)  # in metric ton CO2/acre/year

carbon_offset = sequestered_per_acre_2020

# Display results
print(f"Annual net change in carbon stocks in year 2020: {net_change} MT C")
print(f"Carbon stock density in year 2020: {density_2020} metric tons of carbon stored per hectare")
print(f"Annual net change in carbon stock per area in year 2020: {net_change_per_area_2020} metric tons of carbon sequestered per hectare per year")
print(f"Carbon sequestered annually by one acre of average U.S. forest: {sequestered_per_acre_2020} metric ton CO2/acre/year")


Annual net change in carbon stocks in year 2020: 149000000.0 MT C
Carbon stock density in year 2020: 206.18235062628295 metric tons of carbon stored per hectare
Annual net change in carbon stock per area in year 2020: 0.5282545265031323 metric tons of carbon sequestered per hectare per year
Carbon sequestered annually by one acre of average U.S. forest: 0.7838502919183418 metric ton CO2/acre/year


## 3. Acres of U.S. forests preserved from conversion to cropland in one year

In [11]:
#Source: https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references#pineforests

# Constants
biomass_growth_cropland = 2.25  # mt C/hectare
initial_biomass_stock_forest = -89.84  # mt C/hectare
annual_biomass_loss = 0.0  # mt C/hectare/year (assumed to be zero)

last_year_soil_carbon_stock = 41.13  # mt/hectare
begin_inventory_soil_carbon_stock = 116.0  # mt/hectare
transition_period = 20.0  # years

ef_cropland = 13.17  # mt C/hectare/year
ef_forestland = 2.60 + 0.31  # mt C/hectare/year

# Calculations
# Annual change in carbon stocks in biomass on land converted to cropland
delta_CB = biomass_growth_cropland + initial_biomass_stock_forest - annual_biomass_loss

# Annual change in organic carbon stocks in mineral and organic soils
delta_CSoil = (last_year_soil_carbon_stock - begin_inventory_soil_carbon_stock) / transition_period

# Annual change in emissions from drained organic soils
delta_LOrganic = ef_cropland - ef_forestland

# Total change in carbon density from converting forestland to cropland
total_carbon_loss_hectare = delta_CB + delta_CSoil - delta_LOrganic  # mt C/hectare/year

# Convert to CO2
total_co2_loss_hectare = total_carbon_loss_hectare * (44 / 12)  # mt CO2/hectare/year

# Convert hectare to acre (1 hectare = 2.47105 acres)
total_co2_loss_acre = total_co2_loss_hectare / 2.47105  # mt CO2/acre/year

carbon_offset = -total_co2_loss_acre

# Display results
print(f"Annual change in carbon stocks in biomass: {delta_CB:.2f} mt C/hectare/year")
print(f"Annual change in organic carbon stocks in soils: {delta_CSoil:.2f} mt C/hectare/year")
print(f"Annual change in emissions from drained organic soils: {delta_LOrganic:.2f} mt C/hectare/year")
print(f"Total carbon loss: {total_carbon_loss_hectare:.2f} mt C/hectare/year")
print(f"Total CO2 loss: {total_co2_loss_hectare:.2f} mt CO2/hectare/year")
print(f"Total CO2 loss per acre: {total_co2_loss_acre:.2f} mt CO2/acre/year")
print("Total carbon offset per acre", carbon_offset)

Annual change in carbon stocks in biomass: -87.59 mt C/hectare/year
Annual change in organic carbon stocks in soils: -3.74 mt C/hectare/year
Annual change in emissions from drained organic soils: 10.26 mt C/hectare/year
Total carbon loss: -101.59 mt C/hectare/year
Total CO2 loss: -372.51 mt CO2/hectare/year
Total CO2 loss per acre: -150.75 mt CO2/acre/year
Total carbon offset per acre 150.74947896643127


## 4. Solar Panel

### 1. Retreive Sunroof dataset

### 2. Dataset analysis

### 3. Data Pre-processing Pipeline

### 4. Build Linear Regression model

#### 4-1. Linear Regression

#### 4-2. Ridge Regression

#### 4-3. Lasso Regression

### 5. Build Random Forest model

### 6. Build KNN Model

### Support Vector Machine

### 7. Build Neural Network Model